In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import nest
import parametersets
from mesocircuit import mesocircuit_framework as mesoframe

In [ ]:
# full-scale parameters
name = 'mesocircuit_MAMV1'
custom_params = parametersets.ps_dicts[name]
meso_exp = mesoframe.MesocircuitExperiment(name, custom_params)

In [ ]:
net_dict = meso_exp.circuits[0].net_dict

In [ ]:
net_dict['num_neurons']

In [ ]:
net_dict['indegrees']

In [ ]:
net_dict['ext_indegrees']

In [ ]:
net_dict['bg_rate']

In [ ]:
net_dict['weight_matrix_mean']

In [ ]:
rates_meso = np.array([0.372, 1.413, 1.199, 2.823, 2.281, 3.763, 1.132, 3.515])
frac_rates = rates_meso / net_dict['bg_rate']

tau_syn_ex = net_dict['neuron_params']['tau_syn_ex']
tau_syn_in = net_dict['neuron_params']['tau_syn_in']
tau_syn_default = net_dict['neuron_params']['tau_syn_default']
tau_syn = np.array([[tau_syn_ex/tau_syn_default, tau_syn_in/tau_syn_default] for i in np.arange(4)]).flatten()

weight_tau_syn_ext = net_dict['weight_ext'] * tau_syn_ex / tau_syn_default

# recurrent input from layers L23, L4, and L5
K_ext_L6E_L6I = []
# to L6E (population 6) and L6I (population 7)
for popid in [6, 7]:
    weights_tau_syn_rec = net_dict['full_weight_matrix_mean'][popid][:-1] * tau_syn
    frac_weights_tau_syn = weights_tau_syn_rec / weight_tau_syn_ext

    rec_input = frac_rates * frac_weights_tau_syn * net_dict['indegrees'][popid][:-1]
    K_ext_L6E_L6I.append(net_dict['ext_indegrees'][popid] + np.sum(rec_input[:6]))

K_ext_L6E = np.round(K_ext_L6E_L6I[0]).astype(int)
K_ext_L6I = np.round(K_ext_L6E_L6I[1]).astype(int)

print(net_dict['ext_indegrees'][6:])
print(K_ext_L6E, K_ext_L6I)


In [ ]:
# simulation parameters
T = 1000.                  # simulation time (ms)
dt = 0.1                   # simulation resolution (ms)

N_scaling = 0.01
K_scaling = 1.

# network parameters
N_L6E = int(net_dict['num_neurons'][6] * N_scaling) #4850               # number of neurons in L5E
N_L6I = int(net_dict['num_neurons'][7] * N_scaling) #1065               # number of neurons in L5I
    
Ksyn_L6E_L6E = net_dict['indegrees'][6,6] #int(2038173 / N_L6E)     # number of synapses with presynaptic neuron (pre) in L5E and postsynaptic neuron (post) in L5E 
Ksyn_L6E_L6I = net_dict['indegrees'][7,6] #int(319602 / N_L6I)   # number of synapses with pre in L5E and post in L5I
Ksyn_L6I_L6I = net_dict['indegrees'][7,7] #int(430775 / N_L6I)     # number of synapses with pre in L5I and post in L5I
Ksyn_L6I_L6E = net_dict['indegrees'][6,7] #int(2411184 / N_L6E)     # number of synapses with pre in L5I and post in L5E

K_L6E_ext = K_ext_L6E #2000           # indegree of excitatory neurons to external poisson drive
K_L6I_ext = K_ext_L6I #1900           # indegree of inhibitory neurons to external poisson drive

# neuron parameters
neuron_params = {
    'C_m'       : 250.,    # pF
    'I_e'       : 0.0,     # nA
    'tau_m'     : 10.0,    # ms
    't_ref'     : 2.0,     # ms
    'tau_syn_ex': tau_syn_ex,     # ms
    'tau_syn_in': tau_syn_in,     # ms
    'V_reset'   : -65.0,   # mV
    'E_L'       : -65.0,   # mV
    'V_th'      : -50.0    # mV
}

# synapse parameters
w = net_dict['weight_matrix_mean'][0,0] #87.8                   # mean excitatory weight (pA)
sigma_w = net_dict['weight_matrix_mean'][0,0] * net_dict['weight_rel_std']#8.8              # standard deviation of excitatory weight (pA)
g = net_dict['weight_matrix_mean'][0,1] / net_dict['weight_matrix_mean'][0,0]#-4.                    # relative inhibitory weight 

de = 1.5                   # mean spike transmission delay for excitatory presynaptic neurons (ms)
sigma_de = 0.75            # standard deviation 
di = 0.8                   # mean spike transmission delay for inhibitory presynaptic neurons (ms)
sigma_di = 0.4             # standard deviation
 
# input parameters
bg_rate = 8.               # external poisson rate

# space
extent = net_dict['extent']
beta_L6E_L6E = net_dict['beta'][6,6]
beta_L6E_L6I = net_dict['beta'][7,6]
beta_L6I_L6I = net_dict['beta'][7,7]
beta_L6I_L6E = net_dict['beta'][6,7]
mask_scaling = net_dict['mask_scaling']

In [ ]:
net_dict['beta']

In [ ]:
print(N_L6E, N_L6I)
print(Ksyn_L6E_L6E, Ksyn_L6E_L6I, Ksyn_L6I_L6I, Ksyn_L6I_L6E)
print(w, sigma_w, g)

In [ ]:
nest.ResetKernel()
nest.resolution = dt      # set simulation resolution

In [ ]:
# random positions in 2D with periodic boundary conditions
positions = nest.spatial.free(
    pos=nest.random.uniform(min=-extent / 2.,
                            max=extent / 2.),
    edge_wrap=True,
    extent=[extent, extent])

In [ ]:
# create excitatory and inhibitory neuron populations
pop_L6E = nest.Create('iaf_psc_exp', N_L6E, params=neuron_params, positions=positions)

pop_L6I = nest.Create('iaf_psc_exp', N_L6I, params=neuron_params, positions=positions)

In [ ]:
# excitatory connections (connections from L6E)

# specifying the synapse parameters
syn_dict = {
    'synapse_model': 'static_synapse',
    'weight': nest.math.redraw(
        nest.random.normal(
            mean=w,
            std=sigma_w),
        min=0,
        max=np.Inf),
    'delay': nest.math.redraw(
                nest.random.normal(
            mean=de,
            std=sigma_de),
        min=nest.resolution, # TODO
        max=np.Inf)
}

# connections to L6E
# specifying the connection parameters
conn_dict = {'rule': 'fixed_indegree',
             'indegree': Ksyn_L6E_L6E,
             'p': nest.spatial_distributions.exponential(
                x=nest.spatial.distance,
                beta=beta_L6E_L6E),
              'mask': {'circular': {
                'radius': beta_L6E_L6E * mask_scaling}}}
nest.Connect(pop_L6E, pop_L6E, conn_dict, syn_dict)

# connections to L6I
conn_dict = {'rule': 'fixed_indegree',
             'indegree': Ksyn_L6E_L6I,
             'p': nest.spatial_distributions.exponential(
                x=nest.spatial.distance,
                beta=beta_L6E_L6I),
              'mask': {'circular': {
                'radius': beta_L6E_L6I * mask_scaling}}}
nest.Connect(pop_L6E, pop_L6I, conn_dict, syn_dict)

In [ ]:

# inhibitory connections (connections from L6I)

# specifying the synapse parameters
syn_dict = {
    'synapse_model': 'static_synapse',
    'weight': nest.math.redraw(
        nest.random.normal(
            mean=g*w,
            std=np.abs(g*sigma_w)),
        min=-np.Inf,
        max=0),
    'delay': nest.math.redraw(
                nest.random.normal(
            mean=di,
            std=sigma_di),
        min=nest.resolution, # TODO
        max=np.Inf)
}

# connections to L6E
# specifying the connection parameters
conn_dict = {'rule': 'fixed_indegree',
             'indegree': Ksyn_L6I_L6E,
             'p': nest.spatial_distributions.exponential(
                x=nest.spatial.distance,
                beta=beta_L6I_L6E),
              'mask': {'circular': {
                'radius': beta_L6I_L6E * mask_scaling}}}
nest.Connect(pop_L6I, pop_L6E, conn_dict, syn_dict)

# connections to L6I
conn_dict = {'rule': 'fixed_indegree',
             'indegree': Ksyn_L6I_L6I,
             'p': nest.spatial_distributions.exponential(
                x=nest.spatial.distance,
                beta=beta_L6I_L6I),
              'mask': {'circular': {
                'radius': beta_L6I_L6I * mask_scaling}}}
nest.Connect(pop_L6I, pop_L6I, conn_dict, syn_dict)
     

In [ ]:
# connections with external poisson generator

# connections to L6E
poisson_generator_L6E = nest.Create('poisson_generator',
                                    params={'rate': bg_rate*K_L6E_ext})
nest.Connect(poisson_generator_L6E, pop_L6E, 'all_to_all', {'weight': w, 'delay': de})

# connections to L6I
poisson_generator_L6I = nest.Create('poisson_generator',
                                    params={'rate': bg_rate*K_L6I_ext})
nest.Connect(poisson_generator_L6I, pop_L6I, 'all_to_all', {'weight': w, 'delay': de})

In [ ]:
# set up and connect spike detector
sd_L6E = nest.Create('spike_recorder')
nest.Connect(pop_L6E, sd_L6E, 'all_to_all')

sd_L6I = nest.Create('spike_recorder')
nest.Connect(pop_L6I, sd_L6I, 'all_to_all')

In [ ]:
# run simulation
nest.Simulate(T)    

In [ ]:
# read out recorded spikes
spike_senders_L6E = nest.GetStatus(sd_L6E)[0]['events']['senders']
spike_times_L6E = nest.GetStatus(sd_L6E)[0]['events']['times']
spike_senders_L6I = nest.GetStatus(sd_L6I)[0]['events']['senders']
spike_times_L6I = nest.GetStatus(sd_L6I)[0]['events']['times']

spike_senders = np.hstack((spike_senders_L6E, spike_senders_L6I))
spike_times = np.hstack((spike_times_L6E, spike_times_L6I))

In [ ]:
# plotting
plt.plot(spike_times, spike_senders, 'k.', markersize=1)
plt.xlim(0, T)
plt.ylim(0, N_L6E+N_L6I)
plt.xlabel('time (ms)')
plt.ylabel('neuron id')

In [ ]:
# compute average firing rate
rate_E = nest.GetStatus(sd_L6E)[0]['n_events']/T*1e3/N_L6E
print(f'Firing rate E = {rate_E:.2f} spikes/s')

rate_I = nest.GetStatus(sd_L6I)[0]['n_events']/T*1e3/N_L6I
print(f'Firing rate I = {rate_I:.2f} spikes/s')
     
